In [1]:
import collections

from graphdatascience import GraphDataScience
from torch_geometric.data import Data, download_url
import torch
import torch.optim as optim
from torch_geometric.nn import TransE

In [2]:
gds = GraphDataScience("bolt://localhost:7687", auth=('neo4j', 'neo4jneo4j'), database="ttt")

In [3]:
url = ('https://raw.githubusercontent.com/villmow/datasets_knowledge_embedding/master/FB15k-237')
raw_file_names = ['train.txt', 'valid.txt', 'test.txt']
raw_dir = './data_from_url'
for filename in raw_file_names:
    download_url(f'{url}/{filename}', raw_dir)

Using existing file train.txt
Using existing file valid.txt
Using existing file test.txt


In [4]:
gds.run_cypher("CREATE CONSTRAINT entity_id FOR (e:Entity) REQUIRE e.id IS UNIQUE")

""


In [5]:
rel_types = {
    "train.txt":"TRAIN",
    "valid.txt":"VALID",
    "test.txt":"TEST",
}
rel_id_to_text_dict = {}
rel_type_dict = collections.defaultdict(list)

def process():
    node_dict_, rel_dict_ = {}, {}
    for file_name in raw_file_names:
        file_name_path = raw_dir + '/' + file_name

        with open(file_name_path, 'r') as f:
            data = [x.split('\t') for x in f.read().split('\n')[:-1]]

        list_of_dicts = []
        for i, (src, rel, dst) in enumerate(data):
            if src not in node_dict_:
                node_dict_[src] = len(node_dict_)
            if dst not in node_dict_:
                node_dict_[dst] = len(node_dict_)
            if rel not in rel_dict_:
                rel_dict_[rel] = len(rel_dict_)
                rel_id_to_text_dict[rel_dict_[rel]] = rel

            source = node_dict_[src]
            target = node_dict_[dst]
            edge_type = rel_dict_[rel]

            rel_type_dict[edge_type].append({
                "source":source,
                "target":target,
            })
            list_of_dicts.append({
                "source": source,
                "source_text": src,
                "target": target,
                "target_text": dst,
                "rel_id": edge_type,
            })

        rel_type = rel_types[file_name]
        print(f"Writing {len(list_of_dicts)} entities of {rel_type}")
        gds.run_cypher(
                    "UNWIND $ll as l "+
                    "MERGE (n:Entity {id:l.source, text:l.source_text}) "+
                    "MERGE (m:Entity {id:l.target, text:l.target_text}) "+
                    "MERGE (n)-[:"+rel_type+" {rel_id:l.rel_id}]->(m) ",
                    params={
                        "ll": list_of_dicts
                    },
                )

    for rel_id in rel_id_to_text_dict:
        REL_TYPE = f"REL_{rel_id}"
        print(f"Writing {len(rel_id_to_text_dict[rel_id])} entities of {REL_TYPE}")
        gds.run_cypher(
                "UNWIND $ll AS l MATCH (n:Entity {id:l.source}), (m:Entity {id:l.target}) "+
                "MERGE (n)-[:"+REL_TYPE+" {rel_id:$rel_id, text:$text}]->(m) ",
                params={
                    "ll": rel_type_dict[rel_id],
                    "rel_id": rel_id,
                    "text": rel_id_to_text_dict[rel_id]
                },
            )

process()

Writing 272115 entities of TRAIN
Writing 17535 entities of VALID
Writing 20466 entities of TEST


In [ ]:
# Node: (:Entity {id:int, text:str})
# Edge: [:(TRAIN|TEST|VAL) {rel_id:int, text:str}]

In [ ]:
def print_graph_info(G):
    print(f"Graph '{G.name()}' node count: {G.node_count()}")
    print(f"Graph '{G.name()}' node labels: {G.node_labels()}")
    print(f"Graph '{G.name()}' relationship types: {G.relationship_types()}")
    print(f"Graph '{G.name()}' relationship count: {G.relationship_count()}")

def project_graph(edge_label):
    node_projection = {"Entity": {"properties": "id"}}
    relationship_projection = {edge_label : {"orientation": "NATURAL", "properties": "rel_id"}}
    G, result = gds.graph.project(
        "fb15k-graph-tt"+edge_label,
        node_projection,
        relationship_projection,
    )
    print(f"The projection took {result['projectMillis']} ms")
    print_graph_info(G)

    return G

def get_whole_graph():
    node_projection = {"Entity": {"properties": "id"}}
    relationship_projection = {
        "TRAIN" : {"orientation": "NATURAL", "properties": "rel_id"},
        "TEST" : {"orientation": "NATURAL", "properties": "rel_id"},
        "VAL" : {"orientation": "NATURAL", "properties": "rel_id"},
    }
    G, result = gds.graph.project(
        "fb15k-graph-whole",
        node_projection,
        relationship_projection,
    )
    print(f"The projection took {result['projectMillis']} ms")
    print_graph_info(G)

    return G

In [ ]:
train_G = project_graph("TRAIN")
test_G = project_graph("TEST")
val_G = project_graph("VAL")
db_G = get_whole_graph()

In [ ]:
node_properties = gds.graph.nodeProperties.stream(
    db_G,
    ["id"],
    separate_property_columns=True,
)
print(node_properties)

In [ ]:
nodeId_to_id = dict(zip(node_properties.nodeId, node_properties.id))
id_to_nodeId = dict(zip(node_properties.id, node_properties.nodeId))

In [ ]:
def create_data_from_graph(graph):
    rels_tmp = gds.graph.relationshipProperties.stream(graph, ["rel_id"], separate_property_columns=True)
    topology = [rels_tmp.sourceNodeId.map(lambda x: nodeId_to_id[x]), rels_tmp.targetNodeId.map(lambda x: nodeId_to_id[x])]
    edge_index = torch.tensor(topology, dtype=torch.long)
    edge_type = torch.tensor(rels_tmp.rel_id.astype(int), dtype=torch.long)
    data = Data(edge_index=edge_index, edge_type=edge_type)
    data.num_nodes = len(nodeId_to_id)
    display(data)
    return data

train_tensor_data = create_data_from_graph(train_G)
test_tensor_data = create_data_from_graph(test_G)
val_tensor_data = create_data_from_graph(val_G)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = TransE(
    num_nodes=train_tensor_data.num_nodes,
    num_relations=train_tensor_data.num_edge_types,
    hidden_channels=50,
).to(device)

loader = model.loader(
    head_index=train_tensor_data.edge_index[0],
    rel_type=train_tensor_data.edge_type,
    tail_index=train_tensor_data.edge_index[1],
    batch_size=1000,
    shuffle=True,
)

optimizer = optim.Adam(model.parameters(), lr=0.01)


def train():
    model.train()
    total_loss = total_examples = 0
    for head_index, rel_type, tail_index in loader:
        optimizer.zero_grad()
        loss = model.loss(head_index, rel_type, tail_index)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * head_index.numel()
        total_examples += head_index.numel()
    return total_loss / total_examples


@torch.no_grad()
def test(data):
    model.eval()
    return model.test(
        head_index=data.edge_index[0],
        rel_type=data.edge_type,
        tail_index=data.edge_index[1],
        batch_size=20000,
        k=10,
    )


for epoch in range(1, 501):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
    if epoch % 75 == 0:
        rank, hits = test(val_tensor_data)
        print(f'Epoch: {epoch:03d}, Val Mean Rank: {rank:.2f}, '
              f'Val Hits@10: {hits:.4f}')

print(model)
idx = torch.LongTensor([1])
print(model.rel_emb(idx))
rank, hits_at_10 = test(test_tensor_data)
print(f'Test Mean Rank: {rank:.2f}, Test Hits@10: {hits_at_10:.4f}')

In [ ]:
torch.save(model, "./model_501.pt")

In [ ]:
# torch.save(model.state_dict(), "./model_501_st_dict")

In [ ]:
model = torch.load("./model_501.pt")

In [ ]:
# write embeddings to graph
for i in range(0, len(nodeId_to_id)):
    if i % 100 == 0:
        print(f"Node embeddings uploading: {i} of {len(nodeId_to_id)}", end="\r")
    gds.run_cypher(
            "MATCH (n:Entity {id: $i}) SET n.emb = $EMBEDDING",
            params={
                "i": i,
                "EMBEDDING": model.node_emb.weight[i].tolist()
            },
        )

In [ ]:
# prediction stage

In [ ]:
# 1. Pick relation id to predict
rel_id_to_predict = 17
rel_label_to_predict = f"REL_{rel_id_to_predict}"

In [ ]:
# 2. Put correspond relationships to the graph
def write_rel_chunk(ll:list, label):
    gds.run_cypher(
            "UNWIND $list AS l MATCH (e_s:Entity {id: l.source}), (e_t:Entity {id: l.target}) "+
            "CREATE (e_s)-["+label+"{ text: l.text }]->(e_t)",
            params={"list": ll},
        )

data = test_tensor
list_of_target_rels = []
for i in range(data.num_edges):
    if data.edge_type[i].item() != rel_id_to_predict:
        pass
    source = data.edge_index[0, i].item()
    target = data.edge_index[1, i].item()
    id = data.edge_type[i].item()
    text = rel_id_to_text_dict[id]
    l = {
        "source": source,
        "target": target,
        "text": text
    }
    list_of_target_rels.append(l)
write_rel_chunk(list_of_target_rels, ":"+rel_label_to_predict)

In [ ]:
# 3. Get embedding of that rel from the model
target_emb = model.node_emb.weight[rel_id_to_predict].tolist()

In [ ]:
# 4. Project graph to test
node_projection_test = {"Entity": {"properties": ["id", "emb"] }}
relationship_projection_test = {rel_label_to_predict : {"orientation": "NATURAL"}}
G_test, result = gds.graph.project(
        "graph_to_test1",
        node_projection_test,
        relationship_projection_test,
    )
print_graph_info(G_test)

In [ ]:
# 5. Set the model to predict
transe_model = gds.model.transe.create(
    G_test, "emb", {rel_label_to_predict: target_emb}
)
result = transe_model.predict_stream(
    source_node_filter=[id_to_nodeId[5], id_to_nodeId[10]],
    target_node_filter="Entity",
    relationship_type=rel_label_to_predict,
    top_k=3,
    concurrency=4
)
print(result)